#Machine Learning pt.12

---

#Artificia Neural Network

Egy Vedriyanto 2117051035

##Library

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

##Load Data

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/rahmantaufik27/dataset/main/study/churn_bank_customer.csv")
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


##EDA

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [4]:
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


##Preprocessing

In [5]:
#split data menjadi data X dan Y 
#data exited digunakan sebagai data Y
Y = df.iloc[:, -1].values
#selain data Exited, RowNumer, CustomerID, Surname, digunakan sebagai data X
X = df.iloc[:,3:-1].values
print(Y)
print(X)

[1 0 1 ... 1 1 0]
[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [6]:
#Untuk data X, kita ubah terlebih dahulu data categorical (Geography, Gender) menjadi number
le = LabelEncoder()
X[:,2] = np.array(le.fit_transform(X[:,2]))
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(),[1])],remainder="passthrough")
X = np.array(ct.fit_transform(X))
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [7]:
#split data otomatis/random menggunakan fungsi library tertentu
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=0)

In [8]:
#scaling data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

##Modeling

In [9]:
#Buat fungsi untuk pemodelan ANN
model_ann = tf.keras.Sequential([
    tf.keras.layers.Dense(units=6, activation="relu"),
    tf.keras.layers.Dense(units=1, activation="sigmoid")
])

#compile model ann
model_ann.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])
#fitting data dengan model ann yang dibuat
model_ann.fit(X_train, Y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.7557 - accuracy: 0.4929
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.5652 - accuracy: 0.7717
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4741 - accuracy: 0.8083
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4423 - accuracy: 0.8156
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4309 - accuracy: 0.8171
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4244 - accuracy: 0.8180
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4194 - accuracy: 0.8199
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4146 - accuracy: 0.8200
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4101 - accuracy: 0.8195
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4060 - accura



*   Pada proses modeling diatas kita dapat melihat akurasi data training, akurasi tersebut bukanlah evaluasi yang dicari
*   Akurasi akhir yang peru dicari harus mewakili data not-before-seen"/"not-trained" yang mana contohnya dibawah ini



##Evaluation

In [10]:
model_ann.evaluate(X_test, Y_test, batch_size=8)

250/250 [==============================] - 1s 2ms/step - loss: 0.3321 - accuracy: 0.8595


[0.33208203315734863, 0.859499990940094]

In [11]:
#coba prediksi suatu data
print(model_ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 379ms/step
[[False]]
